In [349]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

In [350]:
# Load environment variables from .env file
load_dotenv(os.path.join(os.getcwd(), '..', '.env'))

# Database connection parameters from environment variables
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

# Create a database connection
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Query to fetch data from the database
query_1 = "SELECT * FROM inspection.cv_detections;"
query_2 = "SELECT * FROM ml.inspection_training;"

In [351]:
# Load SQL data into a pandas dataframe using a direct connection
with engine.connect() as connection:
    df_cv = pd.read_sql(text(query_1), connection)

print(f"📊 Data loaded: {df_cv.shape[0]} rows × {df_cv.shape[1]} columns")
print(f"\nFirst few rows:")

📊 Data loaded: 2134 rows × 13 columns

First few rows:


In [352]:
with engine.connect() as connection:
    df_ml = pd.read_sql(text(query_2), connection)

In [353]:
df_cv.head(3)

,detection_id,image_name,shipment_id,container_id,class_id,confidence,bbox_x_center,bbox_y_center,bbox_width,bbox_height,bbox_area,detected_at,model_version
0,c14e129b-62fa-48c7-9d7c-98acedb7f1bf,14_20220428T012931844Z_s00.mp4_108400.jpg,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,CNT-59083db3-904b-4876-992a-3aee14f226d7,0,0.989,0.535185,0.273698,0.035185,0.197396,0.006945,2026-01-26 18:37:47.802469,yolo_v8
1,b73784e2-1036-4d72-9b7c-e3662afbaeb4,14_20220428T012931844Z_s00.mp4_108400.jpg,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,CNT-59083db3-904b-4876-992a-3aee14f226d7,0,0.987,0.954630,0.268490,0.029630,0.226562,0.006713,2026-01-26 18:37:47.802495,yolo_v8
2,6dd811b3-6437-4d41-b73e-b513de0d3bee,14_20220428T012931844Z_s00.mp4_108400.jpg,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,CNT-59083db3-904b-4876-992a-3aee14f226d7,1,0.876,0.145833,0.521875,0.084259,0.022917,0.001931,2026-01-26 18:37:47.802498,yolo_v8


In [354]:
df_ml.head(3)

,shipment_id,total_detections,avg_confidence,total_damage_area,dent_count,defect_rate,is_high_risk
0,76ca8527-35b6-4b6e-82a5-ac1516853d14,4,0.8425,0.011481,0,0.226086,0
1,bf046423-d395-4109-8431-e03a6ac08bf5,2,0.8870,0.012859,0,0.232399,1
2,160ad73a-faa7-4cd5-a68d-7c351a0c30a1,2,0.8560,0.005352,0,0.224839,0


In [355]:
set(df_cv["shipment_id"]).intersection(set(df_ml["shipment_id"])) # Checking pipeline logic if set() logic is right, data allignment is missing 

set()

In [356]:
obj_cols = df_cv.select_dtypes(include=['object']).columns
num_cols = df_cv.select_dtypes(include=['int', 'float64']).columns

print(f"data of object columns: {obj_cols}")
print(f"data of numerical columns: {num_cols}")

data of object columns: Index(['detection_id', 'image_name', 'shipment_id', 'container_id', 'class_id',
       'model_version'],
      dtype='object')
data of numerical columns: Index(['confidence', 'bbox_x_center', 'bbox_y_center', 'bbox_width',
       'bbox_height', 'bbox_area'],
      dtype='object')


# Container damage detection to prevent bad shipment
--------
# SQL comprehensive analytics data for proper data:
### * **Insert container images for damage box detection**
### * **Analytics statisctical to ensure the shipments are reliable**
### * **Combine dataframe ml_inspection and cv_detections to ensure the container are damaged**

# CV detections (low-level evidence)
-----
## * Aggregated inspection features (business-level signals)
## * Machine learning model (risk scoring)
## * Supervise learning for classification where container are damaged
## * RAG + combined YOLO to show images (explainability, reasoning, auditability)

## Feature engineering

In [357]:
df_cv.drop(columns=['model_version'], inplace=True)

# Change df['class_id'] to integer type
df_cv['class_id'] = df_cv['class_id'].astype(int)

In [358]:
# Validate data integrity

# CV table checks
assert df_cv.detection_id.is_unique
assert df_cv.confidence.between(0, 1).all()

# ML table checks
assert df_ml.shipment_id.is_unique
assert set(df_ml.is_high_risk.unique()).issubset({0,1})

# Aggregating CV → ML (Reproducible Logic)
----
## This gives you lineage trust:
#### * **ML feature ← CV evidence ← Image**

In [359]:
# Aggregating data detection into data inspection (Machine learning EDA)
cv_agg = (
    df_cv
    .groupby("shipment_id")
    .agg(
        total_detections=("detection_id", "count"),
        avg_confidence=("confidence", "mean"),
        total_damage_area=("bbox_area", "sum"),
        dent_count=("class_id", lambda x: (x ==0 ).sum())
    )
    .reset_index()
)

cv_agg["defect_rate"] = (
    cv_agg["total_damage_area"] / cv_agg["total_detections"]
)

## Combine CV Aggregates with ML Labels

In [360]:
df_final = cv_agg.merge(
    df_ml[["shipment_id", "is_high_risk"]],
    on="shipment_id",
    how="left",
    indicator=True
)
df_final["_merge"].value_counts()

left_only     926
right_only      0
both            0
Name: _merge, dtype: int64

## Insert Container Images (Vision + Tabular)

In [361]:
image_map = (
    df_cv
    .groupby("shipment_id")["image_name"]
    .apply(list)
    .reset_index()
)

df_final = df_final.merge(image_map, on="shipment_id")

## Analytics: Shipment Reliability Metrics


In [362]:
df_final["risk_score"] = (
    df_final["avg_confidence"] *
    df_final["total_damage_area"]
)

df_final["reliability_flag"] = (
    (df_final["risk_score"] < 0.01) &
    (df_final["total_detections"] < 3)
).astype(int)

In [363]:
# Handle missing value with bootstrap labels
df_final["is_high_risk"] = df_final["is_high_risk"].fillna(
    (df_final["risk_score"] > 0.01).astype(int)
)

# change df["is_high_risk"] into integer value

df_final["is_high_risk"] = df_final["is_high_risk"].astype(int)
print(f"Total rows of df_final: {len(df_final)}")

Total rows of df_final: 926


# Retrieval dataset of final dataframe based on:
#### * **Aggretating detection data through inspection data**
#### * **Merged image name to file images**

In [364]:
df_final = df_final.drop(columns=["_merge"])

# Clean and deduplicate images while keeping them grouped by shipment_id
df_images_clean = (
    df_final[["shipment_id", "image_name"]]
    .explode("image_name")
    .copy()
)
df_images_clean["image_name"] = df_images_clean["image_name"].str.strip()
df_images_clean = df_images_clean.drop_duplicates()

# Group back into lists per shipment_id
df_images_grouped = (
    df_images_clean
    .groupby("shipment_id")["image_name"]
    .apply(list)
    .reset_index()
)

# Merge back to df_final
df_final = df_final.drop(columns=["image_name"])
df_final = df_final.merge(df_images_grouped, on="shipment_id", how="left")

# Join list items into a string
df_final["image_name"] = df_final["image_name"].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

In [366]:
df_images = (
    df_cv[["shipment_id", "image_name"]].drop_duplicates()
)
df_images.head()

,shipment_id,image_name
0,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,14_20220428T012931844Z_s00.mp4_108400.jpg
3,df75a457-a3f2-42b3-98fb-650fea877b70,114_20220425T114900821Z_s00.mp4_35100.jpg
6,45d62050-340e-422c-89a2-ab77ecd1f945,109_20220211T041216722Z_s00.mp4_75800.jpg
8,648f4444-df86-42ca-994a-a162adefab53,116_20220322T011330692Z_s00.mp4_23500.jpg
10,e98b5b3a-67f9-4e0d-8eb7-c68558c7a047,10_20220509T035631797Z_s00.mp4___650.jpg


In [373]:
df_cv.head()

,detection_id,image_name,shipment_id,container_id,class_id,confidence,bbox_x_center,bbox_y_center,bbox_width,bbox_height,bbox_area,detected_at
0,c14e129b-62fa-48c7-9d7c-98acedb7f1bf,14_20220428T012931844Z_s00.mp4_108400.jpg,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,CNT-59083db3-904b-4876-992a-3aee14f226d7,0,0.989,0.535185,0.273698,0.035185,0.197396,0.006945,2026-01-26 18:37:47.802469
1,b73784e2-1036-4d72-9b7c-e3662afbaeb4,14_20220428T012931844Z_s00.mp4_108400.jpg,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,CNT-59083db3-904b-4876-992a-3aee14f226d7,0,0.987,0.954630,0.268490,0.029630,0.226562,0.006713,2026-01-26 18:37:47.802495
2,6dd811b3-6437-4d41-b73e-b513de0d3bee,14_20220428T012931844Z_s00.mp4_108400.jpg,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,CNT-59083db3-904b-4876-992a-3aee14f226d7,1,0.876,0.145833,0.521875,0.084259,0.022917,0.001931,2026-01-26 18:37:47.802498
3,f4345e22-343c-41ab-9277-bea38daf2001,114_20220425T114900821Z_s00.mp4_35100.jpg,df75a457-a3f2-42b3-98fb-650fea877b70,CNT-8cebf7ae-ecfd-4484-abce-2a60220a5352,0,0.937,0.561111,0.274740,0.053704,0.226562,0.012167,2026-01-26 18:37:47.802771
4,5eb3a128-9d3a-4d11-81da-2f806b55d278,114_20220425T114900821Z_s00.mp4_35100.jpg,df75a457-a3f2-42b3-98fb-650fea877b70,CNT-8cebf7ae-ecfd-4484-abce-2a60220a5352,0,0.751,0.787500,0.351042,0.032407,0.159375,0.005165,2026-01-26 18:37:47.802774


In [ ]:
# Create features dataframe
df_features = cv_agg.merge(
    df_ml[["shipment_id", "is_high_risk"]],
    on="shipment_id",
    how="left"
)

# Added risk_score in df_features
df_features["risk_score"] = (
    df_features["avg_confidence"] *
    df_features["total_damage_area"]
)

df_features["reliability_flag"] = (
    (df_features["risk_score"] < 0.01) &
    (df_features["total_detections"] < 3)
).astype(int)

# Handle missing value with bootstrap labelss
df_features["is_high_risk"] = df_features["is_high_risk"].fillna(
    (df_features["risk_score"] > 0.01).astype(int)
)

# change df["is_high_risk"] into integer value

df_features["is_high_risk"] = df_features["is_high_risk"].astype(int)
df_features.head()

,shipment_id,total_detections,avg_confidence,total_damage_area,dent_count,defect_rate,is_high_risk,risk_score,reliability_flag
0,005fee03-03d7-4aa4-8f84-8b13018b2877,2,0.8940,0.007224,1,0.003612,0,0.006458,1
1,0079a599-b9f6-4fff-a74c-2cad09b1142e,2,0.8355,0.009506,1,0.004753,0,0.007942,1
2,0085237f-e8dc-48bf-984f-c1bc28e5bcd5,2,0.8375,0.006832,1,0.003416,0,0.005721,1
3,008a8e02-1738-4a2a-9029-6e22aae02c98,3,0.8460,0.016585,2,0.005528,1,0.014031,0
4,00a3d0d4-a430-49ae-8cd8-d838ea5a0059,2,0.8540,0.010993,1,0.005497,0,0.009388,1


In [368]:
# Save relevant data to parquet

#df_features.to_parquet("df_features.parquet", index=False)
#df_images_2.to_parquet("df_images.parquet", index=False)

## I have 3 data frames based on EDA

### * **Data Final**
### * **Data Features**
### * **Data Images**

In [369]:
print(f"Data final: {len(df_final)}")
print(f"Data final: {len(df_features)}")
print(f"Data final: {len(df_images)}")

Data final: 926
Data final: 926
Data final: 947


In [372]:
df_final.head()

,shipment_id,total_detections,avg_confidence,total_damage_area,dent_count,defect_rate,is_high_risk,risk_score,reliability_flag,image_name
0,005fee03-03d7-4aa4-8f84-8b13018b2877,2,0.8940,0.007224,1,0.003612,0,0.006458,1,121_20220503T061259983Z_s00.mp4___1450.jpg
1,0079a599-b9f6-4fff-a74c-2cad09b1142e,2,0.8355,0.009506,1,0.004753,0,0.007942,1,13_20220427T113952428Z_s00.mp4_5000.jpg
2,0085237f-e8dc-48bf-984f-c1bc28e5bcd5,2,0.8375,0.006832,1,0.003416,0,0.005721,1,14_20220428T021130880Z_s00.mp4_59800.jpg
3,008a8e02-1738-4a2a-9029-6e22aae02c98,3,0.8460,0.016585,2,0.005528,1,0.014031,0,14_20220428T012931844Z_s00.mp4_30000.jpg
4,00a3d0d4-a430-49ae-8cd8-d838ea5a0059,2,0.8540,0.010993,1,0.005497,0,0.009388,1,100_20220321T021355981Z_s00.mp4_72700.jpg


In [370]:
df_features.head()

,shipment_id,total_detections,avg_confidence,total_damage_area,dent_count,defect_rate,is_high_risk,risk_score,reliability_flag
0,005fee03-03d7-4aa4-8f84-8b13018b2877,2,0.8940,0.007224,1,0.003612,0,0.006458,1
1,0079a599-b9f6-4fff-a74c-2cad09b1142e,2,0.8355,0.009506,1,0.004753,0,0.007942,1
2,0085237f-e8dc-48bf-984f-c1bc28e5bcd5,2,0.8375,0.006832,1,0.003416,0,0.005721,1
3,008a8e02-1738-4a2a-9029-6e22aae02c98,3,0.8460,0.016585,2,0.005528,1,0.014031,0
4,00a3d0d4-a430-49ae-8cd8-d838ea5a0059,2,0.8540,0.010993,1,0.005497,0,0.009388,1


In [371]:
df_images.head()

,shipment_id,image_name
0,2ad1dfa0-0791-4e07-b12c-9f5b268b6f66,14_20220428T012931844Z_s00.mp4_108400.jpg
3,df75a457-a3f2-42b3-98fb-650fea877b70,114_20220425T114900821Z_s00.mp4_35100.jpg
6,45d62050-340e-422c-89a2-ab77ecd1f945,109_20220211T041216722Z_s00.mp4_75800.jpg
8,648f4444-df86-42ca-994a-a162adefab53,116_20220322T011330692Z_s00.mp4_23500.jpg
10,e98b5b3a-67f9-4e0d-8eb7-c68558c7a047,10_20220509T035631797Z_s00.mp4___650.jpg
